# Uber Data Engineering on GCP

1. On GCP, create a new project `uber-de-project`
2. On the created project, go to Cloud Storage -> Create a bucket with unique name, mine is `uber-data-hieu43`; Do not tick "Enforce public access prevention" -> Upload file `uber_data.csv`
3. Go to tab "Permissions" -> Switcch to fine-grained -> Fine-grained -> Save
4. On file `uber_data.csv` -> Edit access -> ADD ENTRY -> Public, allUser, Reader -> Save

   <img src="images/1.png" width=1500>

5. Go to Compute Engine -> Create instance `uber-de-engine` -> Choose `e2-standard-4` -> Create

   <img src="images/2.png" width=900>

6. Connect to that instance via ssh -> Run following commands:
   ```bash
   sudo apt update -y
   sudo apt install -y python3-pip
   sudo apt install -y python3-venv
   python3 -m venv venv
   source venv/bin/activate
   pip install pandas mage-ai google-cloud google-cloud-bigquery db-dtypes
   pip install scikit-learn==1.3.2
   mage start uber-de-mage
   ```
   You will see "Mage is running at http://localhost:6789"
7. Go to inside the VM instance -> Scroll down to Network interfaces -> Go to inside the interface `nic0` -> You will see current firewall rules
8. Go to tab Firewall -> Create firewall rule -> Set Name to `mage-access`, Targets to "All instances", Source IPv4 ranges to `0.0.0.0/0`; Select "Specified protocols and ports", tick TCP, fill `6789` -> Create
9. Go back to Network interfaces -> Access the External IP, port `6789` (mine is `34.59.230.175:6789`) -> You will access Mage UI

10. Log in with email `admin@admin.com` and password `admin` -> New pipeline -> Standard -> Set Name to `uber_etl` -> Create

    <img src="images/3.png" width=1200>

11. Click "Data loader" -> Python -> API -> Set name to `load_uber_data` -> Save and add -> Write code following the file `extract.py`, copy and paste url of file `uber_data.csv` in Cloud Storage (mine is `https://storage.googleapis.com/uber-data-hieu43/uber_data.csv`) -> Run the block

    <img src="images/4.png" width=1500>

12. Click "Transformer" -> Python -> Generic -> Set name to `uber_transformation` -> Save and add -> Write code following the file `transform.py` -> Run the block
13. Click "Data exporter" -> Python -> Google Big Query -> Set name to `uber_bq_load`

14. On GCP, go to APIs & Services -> Credentials -> Create credentials -> Service account -> Set name to `uber_de_service`, role to "BigQuery Admin" -> Done
15. Go to created service account -> Keys -> Add key -> Create new key -> JSON -> Create -> You will get a JSON file
16. Go back to Mage UI -> Open file `io_config.yaml` -> Change `GOOGLE_SERVICE_ACC_KEY` following the credentials JSON file
17. Go to BigQuery -> Click $\vdots$ on the current project -> Create dataset -> Set ID to `uber_de_dataset`, select Multi-region -> Create dataset
18. Go back to Mage UI, in block `uber_bq_load`, write code following file `load.py`, change `table_id` to `uber-de-project-<id>.uber_de_dataset` -> Run the block

    <img src="images/5.png" width=1500>

19. Go to the created dataset in Big Query -> Check all created tables -> Run queries on your own -> Run query in `analytic_query.sql` -> Table `tbl_analytics` will be created

    <img src="images/6.png" width=1500>

20. Go to Looker Studio -> Watch videos from `1:31:10`